In [1]:
import os, glob, h5py, pyproj
import numpy as np
import matplotlib.pyplot as plt
import pointCollection as pc
%matplotlib widget

Assuming the data has been processed already, and the hdf5 files are somewhere

In [2]:
map_data_path = '/Users/grace/Dropbox/Cornell/projects/003/FIS_data'
data_out_path = '/Users/grace/Dropbox/Cornell/projects/003/result_20200723_with_measures/'
plots_path = '/Users/grace/Dropbox/Cornell/projects/003/plots_20200724/'

In [3]:
out_files = glob.glob(data_out_path + '*hdf5')

Plot just rgt 0848 for now:

In [4]:
data_out_path + 'rgt0848_veloc0.hdf5'

'/Users/grace/Dropbox/Cornell/projects/003/result_20200723_with_measures/rgt0848_veloc0.hdf5'

In [5]:
!h5ls /Users/grace/Dropbox/Cornell/projects/003/result_20200723_with_measures/rgt0848_veloc0.hdf5

along_track_step         Dataset {SCALAR}
contributing_cycles      Dataset {SCALAR}
dx                       Dataset {SCALAR}
gt1l                     Group
gt1r                     Group
gt2l                     Group
gt2r                     Group
gt3l                     Group
gt3r                     Group
max_percent_nans         Dataset {SCALAR}
process_date             Dataset {SCALAR}
product                  Dataset {SCALAR}
search_width             Dataset {SCALAR}
segment_length           Dataset {SCALAR}
smoothing                Dataset {SCALAR}
smoothing_window_size    Dataset {SCALAR}


In [6]:
file = data_out_path + 'rgt0848_veloc0.hdf5'

file = data_out_path + 'rgt0848_veloc0.hdf5'

rgt = '0848'
beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']

correlation_threshold = 0.6

if glob.glob(file):
    print(file)
    
    ### MOA parameters
    moa_datapath = map_data_path
#     spatial_extent = np.array([-102, -76, -98, -74.5])
#     spatial_extent = np.array([-65, -86, -55, -81])
#     spatial_extent = np.array([-65, -86, -55, -81]) # lon, lat, lon, lat
#     lat=spatial_extent[[1, 3, 3, 1, 1]]
#     lon=spatial_extent[[2, 2, 0, 0, 2]]
    
    lat_bounds = [-87, -80]
    lon_bounds = [-65, -55]
    lat = lat_bounds
    lon = lon_bounds
    x_bounds = [-830000, -480000]
    y_bounds = [200000, 580000]
    
    # project the coordinates to Antarctic polar stereographic
    xy=np.array(pyproj.Proj(3031)(lon, lat))
    # get the bounds of the projected coordinates 
    XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
    YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
    MOA=pc.grid.data().from_geotif(os.path.join(moa_datapath, 'moa_2009_1km.tif'), bounds=[XR, YR])

    epsg=3031 #PS?

    plt.close('all')
    fig = plt.figure(figsize=[9,11])
    grid = plt.GridSpec(11, 10, wspace=0.4, hspace=0.2)

    haxMOA=fig.add_subplot(grid[0:4,3:7])

    MOA.show(ax=haxMOA,cmap='gray', clim=[14000, 17000])
    haxMOA.set_xlim(x_bounds)
    haxMOA.set_ylim(y_bounds)
    
    # change labels to km
    xticks = haxMOA.get_xticks()
    newlabels = [int(tick/1e3) for tick in xticks]
    haxMOA.set_xticklabels(newlabels)
    haxMOA.set_xlabel('PS X (km)')

    yticks = haxMOA.get_yticks()
    newlabels = [int(tick/1e3) for tick in yticks]
    haxMOA.set_yticklabels(newlabels)
    haxMOA.set_ylabel('PS Y (km)')
    
    with h5py.File(file, 'r') as f:
            for ib, beam in enumerate(beams):
                hax0=fig.add_subplot(grid[ib+5,0:9])
#                 hax0.set_facecolor((0.75,0.75,0.75))

                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                coeffs = f[f'/{beam}/correlation_coefficients'][()]
                velocs = f[f'/{beam}/velocities'][()]
                v_along=f[f'/{beam}/Measures_v_along'][()]
                xy=np.array(pyproj.proj.Proj(3031)(lons,lats))

                ixs0 = coeffs <= correlation_threshold
                ixs = coeffs > correlation_threshold



                h0 = hax0.scatter(xy[0], velocs, 3, coeffs, vmin = 0, vmax = 1,cmap = 'viridis')
                h1 = hax0.plot(xy[0], v_along, 'k-', linewidth = 1)

                hax0.set_ylim(-150,850)

                h2 = haxMOA.scatter(xy[0][ixs0], xy[1][ixs0], 0.02, 'k')
                h3 = haxMOA.scatter(xy[0][ixs], xy[1][ixs], 0.15, velocs[ixs], vmin = -150, vmax = 700,cmap = 'plasma')

                if ib != len(beams) -1:
                    hax0.set_xticklabels([])
                elif ib == len(beams) -1:
                    xticks = hax0.get_xticks()
                    newlabels = [int(tick/1e3) for tick in xticks]
                    hax0.set_xticklabels(newlabels)
                    hax0.set_xlabel('PS X (km)')
                    
                if ib == 2:
                    hax0.set_ylabel('m / yr along track')
             
                xlims = hax0.get_xlim()
                xloc = xlims[0] +  0.02 * (xlims[1] - xlims[0])
                hax0.text(xloc, 500, beam)
                
                
    # make velocity colorbar
    haxCbar1=fig.add_subplot(grid[0:4, 8:9])
    haxCbar1.axis('off')
    c = plt.colorbar(h3, ax = haxCbar1)
    c.set_label('Along-track Velocity (m/yr)')     
    
    # make correlation coefficient colorbar
    haxCbar2=fig.add_subplot(grid[4:10,9])
    haxCbar2.axis('off')
    c = plt.colorbar(h0, ax = haxCbar2)
    c.set_label('Correlation Coefficient')                        
                
    outfile = plots_path + 'rgt' + rgt + '.' + beam + '_vs_measures.png'
    plt.savefig(outfile, dpi = 300)
#     plt.close('all')

/Users/grace/Dropbox/Cornell/projects/003/result_20200723_with_measures/rgt0848_veloc0.hdf5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-891950., -295950.,  137825.,  623825.]), 'origin': 'lower'}
